In [2]:
import pickle
import pygame
import png
from ipynb.fs.full.Cube import cube
import numpy as np
import random as rand
from ipynb.fs.full.Fish import SimpleFish
from ipynb.fs.full.Fish import Mamo

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.backends.backend_agg as agg
import pylab
import math
class util:
    def __init__(self):
        self.tideLineOffset = 0

    
    def makeOutline(surface, size):
        width = surface.get_width()
        height = surface.get_height()
        tray = pygame.Surface((width+2*size, height+2*size))
        tray.blit(surface, (size, size))
        width = tray.get_rect().width
        height = tray.get_rect().height
        pygame.draw.line(tray, (0,0,0), (size,size), (width-size,size), size)
        pygame.draw.line(tray, (0,0,0), (size,size), (size,height-size), size)
        pygame.draw.line(tray, (0,0,0), (size,height - size), (width-size,height-size), size)
        pygame.draw.line(tray, (0,0,0), (width-size,size), (width-size,height-size), size)  
        
        return tray
        
    def makeText(text, size):
        font = pygame.font.SysFont("chivo.ttf", size)
        begin = font.render(text, True, (0,0,0))
        return begin
    
    def save_object(obj, filename):
        with open(filename, 'wb') as output:  # Overwrites any existing file.
            pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
            

    def makeMaps(mini, pixelMap, gm, sand, width, rows):
        numTideFish = 5
        s = mini.asRGB8()
        mapWidth = s[0]
        mapHeight = s[1]
        pmMap = np.vstack(map(np.uint8, list(s[2])))
        rockPosList = []
        tideKeys = {"left": 9999999, "top": 9999999, "bottom": -1, "right": -1}
        counter = 0
        colRand = [(0, 170, 214),(57, 123, 226),(88, 175, 237),(57, 227, 230)]
        miniMap = pygame.Surface((mapWidth*3 + 10, mapHeight*3 + 10))
        miniMapHeight = mapWidth*3 + 10
        lastPixel = ""
        for e, row in enumerate(pmMap):
            miniI = 0
            c = 0
            starting = (0, 8 * e)
            while len(row) > c:
                pixel = (row[c], row[c + 1], row[c + 2])
                
                tideShift = False
                #TIDESHIFT EDGES
                if pixel == (247, 252, 98):
                    pixel = (1, 213, 255)
                    tideShift = True
                    
                miniLocation = (miniI , e )
                miniCube = cube(miniLocation, mapWidth*3, 55, pixel)
                miniCube.draw(miniMap)
                if pixel == (51, 51, 51) or pixel == (50,50,50) or pixel == (66,66,66) or pixel == (55,55,55): #BROWN
                    rockPosList.append((starting[0], starting[1]))
                    pixel = lastPixel
    #                 1) add the position of said pixel to a list in gm
    #                 2) at the end of the rendering, loop through this list
    #                     2.1) "place" the rock at the "center" for this piece
                elif pixel == (255, 38, 0): #RED
                    tideKeys["left"] = min(tideKeys["left"], starting[0])
                    tideKeys["top"] = min(tideKeys["top"], starting[1])
                    tideKeys["right"] = max(tideKeys["right"], starting[0])
                    tideKeys["bottom"] = max(tideKeys["bottom"], starting[1])
                    pixel = (0, 213, 255)
                
                #TIDESHIFT shallows3
#                 if  pixel == (1, 213, 255) or pixel == (0, 122, 170):
#                     if rand.randint(0,1) == 0:
#                         tideShift = True
                        
                file = pixelMap[pixel]
                r = png.Reader("graphicsPM/{}".format(file))
                s = r.asRGB8()

                img2d = np.vstack(map(np.uint8, list(s[2])))
                
                #Add cude to gm.tideShift
                if tideShift:
#                     gm.tideShift.insert(rand.randint(0,len(gm.tideShift)), {"img2d": img2d, 'starting': starting})
                    gm.tideShift.append({"img2d": img2d, 'starting': starting})
                    
                if file not in sand:
                    gm.water[miniLocation] = {"Salinity": 35}

                for j, elemRow in enumerate(img2d):
                    k = 0
                    i = 0
                    while len(elemRow) > k:
                        color = (elemRow[k], elemRow[k + 1], elemRow[k + 2])
                        if color == (255, 57, 214): #PINK
                            color = colRand[rand.randint(0, 3)]
                        location = (starting[0] + i, starting[1] + j)
                        k = k + 3
                        i = i + 1
                        kind = "WATER"
                        if file in sand:
                            kind = "SAND"
                        cu = cube(location, width, rows, color, kind)
                        gm.add(cu)
                        

                            
                        counter = counter + 1
                starting = (starting[0] + 8, starting[1])
                c = c + 3
                lastPixel = pixel
                miniI = miniI + 1
        gm.rockList = rockPosList
        tempKeys = {}
        tempKeys["TL"] = (tideKeys["left"],tideKeys["top"])
        tempKeys["BL"] = (tideKeys["left"],tideKeys["bottom"])
        tempKeys["TR"] = (tideKeys["right"],tideKeys["top"])
        tempKeys["BR"] = (tideKeys["right"],tideKeys["bottom"])
        tideHeight = tempKeys["BR"][1] - tempKeys["TR"][1]
        tideFish = {}
        first = True
        for c,i in enumerate(range(numTideFish)):
            startingX = min(tempKeys["TL"][0] +c*7, tempKeys["TR"][0])
            startingY = rand.randint((tempKeys["TL"][1] + (6 *( tideHeight // 7))), tempKeys["BR"][1] + 4)
            startingPos = (startingX, startingY)
            endingPos = (startingX, rand.randint(tempKeys["TL"][1], tempKeys["TL"][1] + tideHeight//5))
            fi = Mamo(endingPos)
            fi.toSwimTo = startingPos
            fi.pos = startingPos
#             if first:
#                 fi.debug = True
#                 first = False
            gm.tideFish[endingPos] = fi
        gm.tideKeys = tempKeys
        gm.addRocks()        

        pygame.draw.line(miniMap, (0,0,0), (0,0), (mapWidth*3,0), 3)
        pygame.draw.line(miniMap, (0,0,0), (0,0), (0, mapWidth*3), 3)
        pygame.draw.line(miniMap, (0,0,0), (0,mapWidth*3), (mapWidth*3, mapWidth*3), 3)
        pygame.draw.line(miniMap, (0,0,0), (mapWidth*3,0), (mapWidth*3, mapWidth*3), 3)
        return miniMap
    
    
    def makeMiniMap(row, miniMap):
        m = (pygame.image.load('{}'.format(miniMap)).convert_alpha())
        m = pygame.transform.scale(m, (55*3,56*3))
        mt = util.makeOutline(m, 3)
        
        return mt
        
#         mini = png.Reader("{}".format(miniMap))
#         s = mini.asRGB8()
#         mapWidth = s[0]
#         mapHeight = s[1]
#         pmMap = np.vstack(map(np.uint8, list(s[2])))
#         miniMap = pygame.Surface((mapWidth*3 + 10, mapHeight*3 + 10))
#         for e, row in enumerate(pmMap):
#             miniI = 0
#             c = 0
#             while len(row) > c:
#                 pixel = (row[c], row[c + 1], row[c + 2])
#                 miniLocation = (miniI , e )
#                 miniCube = cube(miniLocation, mapWidth*3, 55, pixel)
#                 miniCube.draw(miniMap)
#                 c = c + 3
#                 miniI = miniI + 1
#         pygame.draw.line(miniMap, (0,0,0), (0,0), (mapWidth*3,0), 3)
#         pygame.draw.line(miniMap, (0,0,0), (0,0), (0, mapWidth*3), 3)
#         pygame.draw.line(miniMap, (0,0,0), (0,mapWidth*3), (mapWidth*3, mapWidth*3), 3)
#         pygame.draw.line(miniMap, (0,0,0), (mapWidth*3,0), (mapWidth*3, mapWidth*3), 3)
#         return miniMap
    
    def makeUnitSelect(loko, butToggle):
        buttons = []
        if loko == "Heeia":
#         buttons.ap pend(pygame.image.load('graphicsPM/rockbutton.png').convert_alpha())
            buttons.append(pygame.image.load('graphicsPM/makahabutton.png').convert_alpha())
            buttons.append(pygame.image.load('graphicsPM/sensorBut.png').convert_alpha())
#         buttons.append(pygame.image.load('graphicsPM/water1.png').convert_alpha())
#         units = ["ROCK", "MAKAHA", "SENSOR", "KAI"]
        units = ["MAKAHA","SENSOR"]
        
        butts = []
        retButts = []

        for i in buttons:
            butts.append(pygame.transform.scale(i, (40,40)))


        height = len(butts)*40 + (len(butts)+1)*10
        tray = pygame.Surface((50,height))
        tray.fill((225, 230, 226))
        for c, i in enumerate(butts):
            thing = tray.blit(i, (5,(c+1)*10 + c*40))
            retButts.append(thing)
            if units[c] == butToggle:
                pygame.draw.line(tray, (50, 168, 82), (5,(c+1)*10 + c*40), (45,(c+1)*10 + c*40), 3)
                pygame.draw.line(tray, (50, 168, 82), (5,(c+1)*10 + c*40), (5,(c+1)*10 + (c+1)*40), 3)
                pygame.draw.line(tray, (50, 168, 82), (5,(c+1)*10 + (c+1)*40), (45,(c+1)*10 + (c+1)*40), 3)
                pygame.draw.line(tray, (50, 168, 82), (45,(c+1)*10 + c*40), (45,(c+1)*10 + (c+1)*40), 3)
        pygame.draw.line(tray, (0,0,0), (0,0), (50,0), 3)
        pygame.draw.line(tray, (0,0,0), (0,0), (0,height), 3)
        pygame.draw.line(tray, (0,0,0), (0,height), (50,height), 3)
        pygame.draw.line(tray, (0,0,0), (50,0), (50,height), 3)
        return tray
    
    def makeTideDash(data):
        dat = data.copy()
        dat
        fig = pylab.figure(figsize=[2.5, 2], # Inches
                           dpi=100,        # 100 dots per inch, so the resulting buffer is 400x400 pixels
                           )
        ax = fig.gca()
        ax.plot(dat)
        ax.set_title("Tide Data")
        ax.set_xlabel('Time')
        ax.set_ylabel("Height (ft)")
        fig.tight_layout()
        canvas = agg.FigureCanvasAgg(fig)
        canvas.draw()

        renderer = canvas.get_renderer()
        raw_data = renderer.tostring_rgb()

        size = canvas.get_width_height()

        tray = pygame.image.fromstring(raw_data, size, "RGB")
        tray = util.makeOutline(tray, 3)
        return tray
    
    def alterSpeed(curSpeed):
        spd = (curSpeed * 100)  / 1
        spd = str(round(spd,2)) + "%"
        speed = (pygame.image.load('graphicsPM/speed.png').convert_alpha())
        speed = pygame.transform.scale(speed, (175,50))
        text = util.makeText(spd, 20)
        tray = pygame.Surface((175,80))
        tray.fill((225, 230, 226, 100))
        tray.blit(speed, (0,20))
        tray.blit(text, (175//2 - text.get_width()//2, 5) )
        speed = util.makeOutline(tray, 3)
        return speed
    
    def graphicsTide(self, start, surface, tideStatus = None):
        speed = (pygame.image.load('graphicsPM/tideMakaha.png').convert())
        speed = pygame.transform.scale(speed, (speed.get_width()*5,speed.get_height()*5))
        width = speed.get_width()
        realStart = 0
        

        
#         if tideStatus == comingIn:
#             delt = 4
#             tideIn = True
#         if tideStatus == goingOut:
#             delt = -4
#             tideOut = True

        if "TL" in start:
            realStart = start["TL"]
        elif "TR" in start:
            realStart = (start["TR"][0] - width, start["TR"][1])
        elif "BL" in start:
            realStart = (start["BL"][0], start["BL"][1] - 8*8*5)
        elif "BR" in start:
            realStart = (start["BL"][0] - width, start["BL"][1] - 8*8*5)
        
        bottom = realStart[1] + 8*8*5
        
        if self.tideLineOffset > bottom - realStart[1]:
            self.tideLineOffset = 0
        elif self.tideLineOffset < 0:
            self.tideLineOffset = bottom - realStart[1]
        
        for i in range(8): 
            newY = realStart[1] + i * 8 * 5 + self.tideLineOffset
            if newY > bottom:
                newY = newY - (bottom - realStart[1])
            surface.blit(speed, (realStart[0], newY))
        if tideStatus < 0: #decreasing
            self.tideLineOffset -= 5
        else:
            self.tideLineOffset += 5

#         pass
    def defineEnclosure(filename, gm):
        baseMap = png.Reader("{}".format(filename))
        s = baseMap.asRGB8()
        mapWidth = s[0]
        mapHeight = s[1]
        pmMap = np.vstack(map(np.uint8, list(s[2])))
        r = png.Reader("graphicsPM/{}".format("water1.png"))
        zz = r.asRGB8()

        img2d = np.vstack(map(np.uint8, list(zz[2])))
        ret = []
        for e, row in enumerate(pmMap):
            miniI = 0
            c = 0
            starting = (0, 8 * e)
            while len(row) > c:
                pixel = (row[c], row[c + 1], row[c + 2])
                #RED
                if pixel == (255, 38, 0):
                    miniLocation = (miniI , e )
                    miniCube = cube(miniLocation, mapWidth*3, 55, pixel)
                    for j, elemRow in enumerate(img2d):
                        k = 0
                        i = 0
                        while len(elemRow) > k:
                            location = (starting[0] + i, starting[1] + j)
                            k = k + 3
                            i = i + 1
                            ret.append(location)
                starting = (starting[0] + 8, starting[1])
                c = c + 3
                miniI = miniI + 1
        return ret
    def connectMakaha(makahaTracker, gm, steps):
#         If first X < second X, go right
#         If second x < first X, go left
        first = makahaTracker[0]["poss"]
        second = makahaTracker[1]["poss"]
        if first[0] < second[0]:
            pass
        if first[0] > second[0]:
            hold = second
            second = first
            first = hold
            
        height = second[1] - first[1]
        width =  second[0] - first[0]
        
        c = math.hypot((height//8),(width//8))
        xDelt = round((width/c) * .7)
        yDelt = round((height/c) * .7)
        curX = first[0]
        curY = first[1]
        curV = makahaTracker[0]["curView"]
        if first[0] != second[0]:
            while curX < second[0]:
                gm.click((curX, curY), curV, "MAKAHA")
#                 makahas.append({"poss": (curX, curY), "curView": curV})
                curX = curX + xDelt
                curY = curY + yDelt
        else:
            while curY < second[1]:
                gm.click((curX, curY), curV, "MAKAHA")
                makahas.append({"poss": (curX, curY), "curView": curV})
                curX = curX + xDelt
                curY = curY + yDelt
        makahaTracker.clear()
        steps["twoClickMakaha"] = False


    def getColor(colorRGBA1, colorRGBA2):
        alpha = 255 - ((255 - colorRGBA1[3]) * (255 - colorRGBA2[3]) / 255)
        red   = (colorRGBA1[0] * (255 - colorRGBA2[3]) + colorRGBA2[0] * colorRGBA2[3]) / 255
        green = (colorRGBA1[1] * (255 - colorRGBA2[3]) + colorRGBA2[1] * colorRGBA2[3]) / 255
        blue  = (colorRGBA1[2] * (255 - colorRGBA2[3]) + colorRGBA2[2] * colorRGBA2[3]) / 255
        return (int(red), int(green), int(blue), int(alpha))


pygame 2.0.0.dev12 (SDL 2.0.12, python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
